<a href="https://colab.research.google.com/github/cjfghk5697/AjouDeeplearning_Compete/blob/main/baseline_CNN_center2(32.4%25%2C%204th).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [22]:
!pip install git+https://github.com/cmpark0126/pytorch-polynomial-lr-decay.git

  Cloning https://github.com/cmpark0126/pytorch-polynomial-lr-decay.git to /tmp/pip-req-build-yqf5jut6
  Running command git clone -q https://github.com/cmpark0126/pytorch-polynomial-lr-decay.git /tmp/pip-req-build-yqf5jut6


In [23]:
!pip3 install timm

In [24]:
import gdown
global PATH
import os
import time
import datetime
import logging
import random
import torch
from torch.autograd import Variable

from google.colab import files
from torch.optim.lr_scheduler import _LRScheduler
from torchsummary import summary
from pathlib import Path
from PIL import Image

import torchvision.models as models

from torch.utils.data.dataset import random_split

import torch.nn as nn
import torch.nn.functional as F
import torchvision.transforms as transforms
from torch.optim.optimizer import Optimizer, required

import pandas as pd
from torchvision.datasets import ImageFolder
from torch.utils.tensorboard import SummaryWriter

import torch.nn.functional as F
from tqdm import tqdm
import argparse
import albumentations as A
import cv2
from glob import glob
from albumentations.core.transforms_interface import ImageOnlyTransform
from matplotlib import pyplot as plt
import math
import timm
import numpy as np
from torch.utils.data import Dataset, DataLoader



url_path_map = {
        'train': {
            'url':'https://drive.google.com/uc?id=1JewlmOsqs-O1EmMMKutcYYiD7GkN6brN', 
            'path':'dataset/train.npy'
            },
        'test': {
            'url':'https://drive.google.com/uc?id=1JcCFZbc_N7VIa3G3XA8grwz4XGdaaXSZ', 
            'path':'dataset/test.npy'
            },
        'label': {
            'url':'https://drive.google.com/uc?id=1JiaTOcZ6QfDThw3RrIa3ZxDDaNt-BMUQ',
            'path':'dataset/label_info.txt'
        }
    }


def download_dataset():
    Path('dataset').mkdir(exist_ok=True)
    for split in ['train', 'test', 'label']:
        if Path(url_path_map[split]['path']).exists():
            continue
        gdown.download(url_path_map[split]['url'], url_path_map[split]['path'], quiet=False)



def read_txt(file_name):
    with open(file_name, 'rt') as f:
        class_list = [row.strip('\n') for row in f.readlines()]
    return class_list


class MyDataset:
    def __init__(self, root, split='train', transform_fn=None, label='label_info'):
        self.dataset = np.load('{}/{}.npy'.format(root, split), allow_pickle=True).item()
        self.label_name = read_txt('{}/{}.txt'.format(root, label))
        self.transform_fn = transform_fn
        self.split = split
    
    @property
    def class_num(self):
        return len(self.label_name)
        
    def __len__(self):
        return len(self.dataset['label'])
    
    def __getitem__(self, idx):
        image, label = self.dataset['image'][idx], self.dataset['label'][idx]
        
        if self.transform_fn:
            image = self.transform_fn(image)
        
        return image, label

In [25]:
def mixup_data(x, y, alpha=1.0, use_cuda=True):
    '''Returns mixed inputs, pairs of targets, and lambda'''
    if alpha > 0:
        lam = np.random.beta(alpha, alpha)
    else:
        lam = 1

    batch_size = x.size()[0]
    if use_cuda:
        index = torch.randperm(batch_size).cuda()
    else:
        index = torch.randperm(batch_size)

    mixed_x = lam * x + (1 - lam) * x[index, :]
    y_a, y_b = y, y[index]
    return mixed_x, y_a, y_b, lam


def mixup_criterion(criterion, pred, y_a, y_b, lam):
    return lam * criterion(pred, y_a) + (1 - lam) * criterion(pred, y_b)

In [26]:
def seed_everything(seed: int = 42):
    random.seed(seed)
    np.random.seed(seed)
    os.environ["PYTHONHASHSEED"] = str(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)  # type: ignore
    torch.backends.cudnn.deterministic = True  # type: ignore
    torch.backends.cudnn.benchmark = True  # type: ignore
seed_everything(42)

In [27]:

class RAdam(Optimizer):

    def __init__(self, params, lr=1e-3, betas=(0.9, 0.999), eps=1e-8, weight_decay=0, degenerated_to_sgd=True):
        if not 0.0 <= lr:
            raise ValueError("Invalid learning rate: {}".format(lr))
        if not 0.0 <= eps:
            raise ValueError("Invalid epsilon value: {}".format(eps))
        if not 0.0 <= betas[0] < 1.0:
            raise ValueError("Invalid beta parameter at index 0: {}".format(betas[0]))
        if not 0.0 <= betas[1] < 1.0:
            raise ValueError("Invalid beta parameter at index 1: {}".format(betas[1]))
        
        self.degenerated_to_sgd = degenerated_to_sgd
        if isinstance(params, (list, tuple)) and len(params) > 0 and isinstance(params[0], dict):
            for param in params:
                if 'betas' in param and (param['betas'][0] != betas[0] or param['betas'][1] != betas[1]):
                    param['buffer'] = [[None, None, None] for _ in range(10)]
        defaults = dict(lr=lr, betas=betas, eps=eps, weight_decay=weight_decay, buffer=[[None, None, None] for _ in range(10)])
        super(RAdam, self).__init__(params, defaults)

    def __setstate__(self, state):
        super(RAdam, self).__setstate__(state)

    def step(self, closure=None):

        loss = None
        if closure is not None:
            loss = closure()

        for group in self.param_groups:

            for p in group['params']:
                if p.grad is None:
                    continue
                grad = p.grad.data.float()
                if grad.is_sparse:
                    raise RuntimeError('RAdam does not support sparse gradients')

                p_data_fp32 = p.data.float()

                state = self.state[p]

                if len(state) == 0:
                    state['step'] = 0
                    state['exp_avg'] = torch.zeros_like(p_data_fp32)
                    state['exp_avg_sq'] = torch.zeros_like(p_data_fp32)
                else:
                    state['exp_avg'] = state['exp_avg'].type_as(p_data_fp32)
                    state['exp_avg_sq'] = state['exp_avg_sq'].type_as(p_data_fp32)

                exp_avg, exp_avg_sq = state['exp_avg'], state['exp_avg_sq']
                beta1, beta2 = group['betas']

                exp_avg_sq.mul_(beta2).addcmul_(1 - beta2, grad, grad)
                exp_avg.mul_(beta1).add_(1 - beta1, grad)

                state['step'] += 1
                buffered = group['buffer'][int(state['step'] % 10)]
                if state['step'] == buffered[0]:
                    N_sma, step_size = buffered[1], buffered[2]
                else:
                    buffered[0] = state['step']
                    beta2_t = beta2 ** state['step']
                    N_sma_max = 2 / (1 - beta2) - 1
                    N_sma = N_sma_max - 2 * state['step'] * beta2_t / (1 - beta2_t)
                    buffered[1] = N_sma

                    # more conservative since it's an approximated value
                    if N_sma >= 5:
                        step_size = math.sqrt((1 - beta2_t) * (N_sma - 4) / (N_sma_max - 4) * (N_sma - 2) / N_sma * N_sma_max / (N_sma_max - 2)) / (1 - beta1 ** state['step'])
                    elif self.degenerated_to_sgd:
                        step_size = 1.0 / (1 - beta1 ** state['step'])
                    else:
                        step_size = -1
                    buffered[2] = step_size

                # more conservative since it's an approximated value
                if N_sma >= 5:
                    if group['weight_decay'] != 0:
                        p_data_fp32.add_(-group['weight_decay'] * group['lr'], p_data_fp32)
                    denom = exp_avg_sq.sqrt().add_(group['eps'])
                    p_data_fp32.addcdiv_(-step_size * group['lr'], exp_avg, denom)
                    p.data.copy_(p_data_fp32)
                elif step_size > 0:
                    if group['weight_decay'] != 0:
                        p_data_fp32.add_(-group['weight_decay'] * group['lr'], p_data_fp32)
                    p_data_fp32.add_(-step_size * group['lr'], exp_avg)
                    p.data.copy_(p_data_fp32)

        return loss

In [28]:
class MnistModel_efficientb7(nn.Module):
    def __init__(self,model, in_channel=30, out_channel=3):
        super().__init__()
        self.model=model
        self.in_channel = in_channel
        self.dropout = nn.Dropout(p=0.2)
        self.conv0 = nn.Conv2d(in_channel, out_channel, kernel_size=(3, 3),padding=(1,1), stride=1)

    def forward(self, x):
        x = self.conv0(x)
        x = self.model(x)

        return x

In [29]:
class MyModelWrapper:
    def __init__(self, log_name, model, criterion, optimizer, scheduler ,device):
        super().__init__()
        self.model = model
        self.criterion = criterion
        self.optimizer = optimizer
        self.scheduler = scheduler

        self.device = device
        self.best_acc = 0
        self.best_loss=100
        self.setup_directory()
        self.log_name = '{}-{}'.format(datetime.datetime.now().strftime('%Y-%m-%d/%H-%M-%S'), log_name)
        self.log_best_weight_path = 'log/best_weight/{}.pth'.format(self.log_name)
        self.writer = SummaryWriter(log_dir='log/tensor_board/{}'.format(self.log_name), filename_suffix=log_name)
        self.log_loss_weight_path = 'log/loss_weight/{}.pth'.format(self.log_name)
    
    def setup_directory(self):
        Path('log/tensor_board/{}'.format(datetime.datetime.now().strftime('%Y-%m-%d'))).mkdir(exist_ok=True, parents=True)
        Path('log/best_weight/{}'.format(datetime.datetime.now().strftime('%Y-%m-%d'))).mkdir(exist_ok=True, parents=True)
        Path('log/loss_weight/{}'.format(datetime.datetime.now().strftime('%Y-%m-%d'))).mkdir(exist_ok=True, parents=True)
    
    def train(self, train_dl, epoch):
        debug_step = len(train_dl)//10
        batch_time = AverageMeter('Time', ':6.3f')
        data_time = AverageMeter('Data', ':6.3f')
        losses = AverageMeter('Loss', ':7.4f')
        top1 = AverageMeter('Acc@1', ':6.2f')
        top5 = AverageMeter('Acc@5', ':6.2f')
        progress = ProgressMeter(
            len(train_dl),
            [batch_time, data_time, losses, top1, top5],
            prefix="TRAIN: [{}]".format(epoch))

        self.model.train()

        end = time.time()
        for step, (x, y) in enumerate(train_dl):
            data_time.update(time.time() - end)
            x, y = x.to(self.device), y.to(self.device)
        
            x, y_a, y_b, lam = mixup_data(x,y)
            x, y_a, y_b = map(Variable, (x, y_a, y_b))
            y_hat = self.model(x)
            loss = mixup_criterion(self.criterion, y_hat, y_a, y_b, lam)

            acc1, acc5 = accuracy(y_hat, y, topk=(1, 5))
            losses.update(loss.item(), x.size(0))
            top1.update(acc1[0], x.size(0))
            top5.update(acc5[0], x.size(0))

            loss.backward()
            self.optimizer.step()
            self.optimizer.zero_grad()

            batch_time.update(time.time() - end)
            end = time.time()

            if step != 0 and step % debug_step == 0:
                progress.display(step)
        return losses.avg, top1.avg

    @torch.no_grad()
    def valid(self, dl):
        debug_step = len(dl) // 10
        batch_time = AverageMeter('Time', ':6.3f')
        losses = AverageMeter('Loss', ':7.4f')
        top1 = AverageMeter('Acc@1', ':6.2f')
        top5 = AverageMeter('Acc@5', ':6.2f')
        progress = ProgressMeter(
            len(dl),
            [batch_time, losses, top1, top5],
            prefix='VALID: ')

        self.model.eval()

        end = time.time()
        for step, (x, y) in enumerate(dl):
            x, y = x.float().to(self.device), y.long().to(self.device)
            y_hat = self.model(x)
            loss = F.cross_entropy(y_hat, y)

            acc1, acc5 = accuracy(y_hat, y, topk=(1, 5))
            losses.update(loss.item(), x.size(0))
            top1.update(acc1[0], x.size(0))
            top5.update(acc5[0], x.size(0))

            batch_time.update(time.time() - end)
            end = time.time()

            if step % debug_step == 0:
                progress.display(step)

        return losses.avg, top1.avg

    
    def fit(self, train_dl,valid_dl, nepoch=40):
        no_improvement=0
        best_acc = 0
        best_acc_arg = 0
        best_loss=100
        
        for epoch in range(nepoch):
            train_loss, train_acc = self.train(train_dl, epoch)
            valid_loss, valid_acc = self.train(valid_dl, epoch)
            flag=0
            if valid_acc > best_acc:
                best_acc = valid_acc
                best_acc_arg = epoch + 1
                self.save_best_weight(self.model, best_acc)
                flag += 1
                no_improvement = 0
            
            
            if valid_loss < best_loss:
                best_loss= valid_loss
                self.save_best_loss_weight(self.model, best_loss)
                flag += 1
                no_improvement = 0
    
            if flag == 0:
              no_improvement += 1
    
            if no_improvement > 30:
              print("No improvement for 3 epochs, stopping")
              break
            self.scheduler.step()
            
            print('=' * 150)
            print(
                '[EPOCH]: {:03d}/{:03d}    train loss {:07.4f} acc {:07.4f}% ('
                'best accuracy : {:07.4f} @ {:03d})'.format(
                    epoch + 1, nepoch, valid_loss, valid_acc, best_acc,
                    best_acc_arg
                ))
            print('=' * 150)
            self.log_tensorboard(epoch, valid_loss, valid_acc)
            
      


    def save_best_weight(self, model, top1_acc):
        global PATH
        if top1_acc > self.best_acc:
            self.best_acc = top1_acc
            print('Saving best model({:07.4f}%) weight to {}'.format(top1_acc, self.log_best_weight_path))
            torch.save({'weight':model.state_dict(), 'top1_acc':top1_acc}, self.log_best_weight_path)
            PATH=self.log_best_weight_path
    
    def save_best_loss_weight(self, model, top1_loss):
        global loss_PATH
        if top1_loss < self.best_loss:
            self.best_loss = top1_loss
            print('Saving best loss model({:07.4f}%) weight to {}'.format(top1_loss, self.log_loss_weight_path))
            torch.save({'weight':model.state_dict(), 'top1_acc':top1_loss}, self.log_loss_weight_path)

            loss_PATH=self.log_loss_weight_path


    def log_tensorboard(self, epoch, train_loss, train_acc):
        self.writer.add_scalar('Loss/train', train_loss, epoch)
        self.writer.add_scalar('Accuracy/train', train_acc, epoch)

In [37]:
class MyModelWrapper_none:
    def __init__(self, log_name, model, criterion, optimizer ,device):
        super().__init__()
        self.model = model
        self.criterion = criterion
        self.optimizer = optimizer

        self.device = device
        self.best_acc = 0
        self.best_loss=100
        self.setup_directory()
        self.log_name = '{}-{}'.format(datetime.datetime.now().strftime('%Y-%m-%d/%H-%M-%S'), log_name)
        self.log_best_weight_path = 'log/best_weight/{}.pth'.format(self.log_name)
        self.writer = SummaryWriter(log_dir='log/tensor_board/{}'.format(self.log_name), filename_suffix=log_name)
        self.log_loss_weight_path = 'log/loss_weight/{}.pth'.format(self.log_name)
    
    def setup_directory(self):
        Path('log/tensor_board/{}'.format(datetime.datetime.now().strftime('%Y-%m-%d'))).mkdir(exist_ok=True, parents=True)
        Path('log/best_weight/{}'.format(datetime.datetime.now().strftime('%Y-%m-%d'))).mkdir(exist_ok=True, parents=True)
        Path('log/loss_weight/{}'.format(datetime.datetime.now().strftime('%Y-%m-%d'))).mkdir(exist_ok=True, parents=True)
    
    def train(self, train_dl, epoch):
        debug_step = len(train_dl)//10
        batch_time = AverageMeter('Time', ':6.3f')
        data_time = AverageMeter('Data', ':6.3f')
        losses = AverageMeter('Loss', ':7.4f')
        top1 = AverageMeter('Acc@1', ':6.2f')
        top5 = AverageMeter('Acc@5', ':6.2f')
        progress = ProgressMeter(
            len(train_dl),
            [batch_time, data_time, losses, top1, top5],
            prefix="TRAIN: [{}]".format(epoch))

        self.model.train()

        end = time.time()
        for step, (x, y) in enumerate(train_dl):
            data_time.update(time.time() - end)
            x, y = x.to(self.device), y.to(self.device)
        
            x, y_a, y_b, lam = mixup_data(x,y)
            x, y_a, y_b = map(Variable, (x, y_a, y_b))
            y_hat = self.model(x)
            loss = mixup_criterion(self.criterion, y_hat, y_a, y_b, lam)

            acc1, acc5 = accuracy(y_hat, y, topk=(1, 5))
            losses.update(loss.item(), x.size(0))
            top1.update(acc1[0], x.size(0))
            top5.update(acc5[0], x.size(0))

            loss.backward()
            self.optimizer.step()
            self.optimizer.zero_grad()

            batch_time.update(time.time() - end)
            end = time.time()

            if step != 0 and step % debug_step == 0:
                progress.display(step)
        return losses.avg, top1.avg

    @torch.no_grad()
    def valid(self, dl):
        debug_step = len(dl) // 10
        batch_time = AverageMeter('Time', ':6.3f')
        losses = AverageMeter('Loss', ':7.4f')
        top1 = AverageMeter('Acc@1', ':6.2f')
        top5 = AverageMeter('Acc@5', ':6.2f')
        progress = ProgressMeter(
            len(dl),
            [batch_time, losses, top1, top5],
            prefix='VALID: ')

        self.model.eval()

        end = time.time()
        for step, (x, y) in enumerate(dl):
            x, y = x.float().to(self.device), y.long().to(self.device)
            y_hat = self.model(x)
            loss = F.cross_entropy(y_hat, y)

            acc1, acc5 = accuracy(y_hat, y, topk=(1, 5))
            losses.update(loss.item(), x.size(0))
            top1.update(acc1[0], x.size(0))
            top5.update(acc5[0], x.size(0))

            batch_time.update(time.time() - end)
            end = time.time()

            if step % debug_step == 0:
                progress.display(step)

        return losses.avg, top1.avg

    
    def fit(self, train_dl,valid_dl, nepoch):
        no_improvement=0
        best_acc = 0
        best_acc_arg = 0
        best_loss=100
        
        for epoch in range(nepoch):
            train_loss, train_acc = self.train(train_dl, epoch)
            valid_loss, valid_acc = self.train(valid_dl, epoch)
            flag=0
            if valid_acc > best_acc:
                best_acc = valid_acc
                best_acc_arg = epoch + 1
                self.save_best_weight(self.model, best_acc)
                flag += 1
                no_improvement = 0

            
            if valid_loss < best_loss:
                best_loss= valid_loss
                self.save_best_loss_weight(self.model, best_loss)
                flag += 1
                no_improvement = 0
    
            if flag == 0:
              no_improvement += 1
    
            if no_improvement > 30:
              print("No improvement for 3 epochs, stopping")
              break
            
            print('=' * 150)
            print(
                '[EPOCH]: {:03d}/{:03d}    train loss {:07.4f} acc {:07.4f}% ('
                'best accuracy : {:07.4f} @ {:03d})'.format(
                    epoch + 1, nepoch, valid_loss, valid_acc, best_acc,
                    best_acc_arg
                ))
            print('=' * 150)
            self.log_tensorboard(epoch, valid_loss, valid_acc)
            
      


    def save_best_weight(self, model, top1_acc):
        global PATH
        if top1_acc > self.best_acc:
            self.best_acc = top1_acc
            print('Saving best model({:07.4f}%) weight to {}'.format(top1_acc, self.log_best_weight_path))
            torch.save({'weight':model.state_dict(), 'top1_acc':top1_acc}, self.log_best_weight_path)
            PATH=self.log_best_weight_path
    
    def save_best_loss_weight(self, model, top1_loss):
        global loss_PATH
        if top1_loss < self.best_loss:
            self.best_loss = top1_loss
            print('Saving best loss model({:07.4f}%) weight to {}'.format(top1_loss, self.log_loss_weight_path))
            torch.save({'weight':model.state_dict(), 'top1_acc':top1_loss}, self.log_loss_weight_path)
            loss_PATH=self.log_loss_weight_path


    def log_tensorboard(self, epoch, train_loss, train_acc):
        self.writer.add_scalar('Loss/train', train_loss, epoch)
        self.writer.add_scalar('Accuracy/train', train_acc, epoch)

In [31]:
class CosineAnnealingWarmupRestarts(_LRScheduler):
    """
        optimizer (Optimizer): Wrapped optimizer.
        first_cycle_steps (int): First cycle step size.
        cycle_mult(float): Cycle steps magnification. Default: -1.
        max_lr(float): First cycle's max learning rate. Default: 0.1.
        min_lr(float): Min learning rate. Default: 0.001.
        warmup_steps(int): Linear warmup step size. Default: 0.
        gamma(float): Decrease rate of max learning rate by cycle. Default: 1.
        last_epoch (int): The index of last epoch. Default: -1.
    """
    
    def __init__(self,
                 optimizer : torch.optim.Optimizer,
                 first_cycle_steps : int,
                 cycle_mult : float = 1.,
                 max_lr : float = 0.1,
                 min_lr : float = 0.001,
                 warmup_steps : int = 0,
                 gamma : float = 1.,
                 last_epoch : int = -1
        ):
        assert warmup_steps < first_cycle_steps
        
        self.first_cycle_steps = first_cycle_steps # first cycle step size
        self.cycle_mult = cycle_mult # cycle steps magnification
        self.base_max_lr = max_lr # first max learning rate
        self.max_lr = max_lr # max learning rate in the current cycle
        self.min_lr = min_lr # min learning rate
        self.warmup_steps = warmup_steps # warmup step size
        self.gamma = gamma # decrease rate of max learning rate by cycle
        
        self.cur_cycle_steps = first_cycle_steps # first cycle step size
        self.cycle = 0 # cycle count
        self.step_in_cycle = last_epoch # step size of the current cycle
        
        super(CosineAnnealingWarmupRestarts, self).__init__(optimizer, last_epoch)
        
        # set learning rate min_lr
        self.init_lr()
    
    def init_lr(self):
        self.base_lrs = []
        for param_group in self.optimizer.param_groups:
            param_group['lr'] = self.min_lr
            self.base_lrs.append(self.min_lr)
    
    def get_lr(self):
        if self.step_in_cycle == -1:
            return self.base_lrs
        elif self.step_in_cycle < self.warmup_steps:
            return [(self.max_lr - base_lr)*self.step_in_cycle / self.warmup_steps + base_lr for base_lr in self.base_lrs]
        else:
            return [base_lr + (self.max_lr - base_lr) \
                    * (1 + math.cos(math.pi * (self.step_in_cycle-self.warmup_steps) \
                                    / (self.cur_cycle_steps - self.warmup_steps))) / 2
                    for base_lr in self.base_lrs]

    def step(self, epoch=None):
        if epoch is None:
            epoch = self.last_epoch + 1
            self.step_in_cycle = self.step_in_cycle + 1
            if self.step_in_cycle >= self.cur_cycle_steps:
                self.cycle += 1
                self.step_in_cycle = self.step_in_cycle - self.cur_cycle_steps
                self.cur_cycle_steps = int((self.cur_cycle_steps - self.warmup_steps) * self.cycle_mult) + self.warmup_steps
        else:
            if epoch >= self.first_cycle_steps:
                if self.cycle_mult == 1.:
                    self.step_in_cycle = epoch % self.first_cycle_steps
                    self.cycle = epoch // self.first_cycle_steps
                else:
                    n = int(math.log((epoch / self.first_cycle_steps * (self.cycle_mult - 1) + 1), self.cycle_mult))
                    self.cycle = n
                    self.step_in_cycle = epoch - int(self.first_cycle_steps * (self.cycle_mult ** n - 1) / (self.cycle_mult - 1))
                    self.cur_cycle_steps = self.first_cycle_steps * self.cycle_mult ** (n)
            else:
                self.cur_cycle_steps = self.first_cycle_steps
                self.step_in_cycle = epoch
                
        self.max_lr = self.base_max_lr * (self.gamma**self.cycle)
        self.last_epoch = math.floor(epoch)
        for param_group, lr in zip(self.optimizer.param_groups, self.get_lr()):
            param_group['lr'] = lr

In [32]:
class AverageMeter(object):
    """Computes and stores the average and current value"""
    def __init__(self, name, fmt=':f'):
        self.name = name
        self.fmt = fmt
        self.reset()

    def reset(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0

    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count

    def __str__(self):
        fmtstr = '{name} {val' + self.fmt + '} ({avg' + self.fmt + '})'
        return fmtstr.format(**self.__dict__)


In [33]:
!CUDA_LAUNCH_BLOCKING=1

In [34]:
os.environ['CUDA_LAUNCH_BLOCKING'] = "1"

In [39]:
def run_vit_large(model_name='vit_large', lr=1e-5, batch_size=8, nepoch=8):

    # step 1. load dataset
    train_transforms = transforms.Compose([
        transforms.ToTensor(),
        transforms.RandomCrop(224),
        transforms.RandomHorizontalFlip(p=0.2),
        transforms.RandomVerticalFlip(p=0.2),
        transforms.RandomRotation(30),
        transforms.Normalize((0.485, 0.456, 0.406)*10, (0.229, 0.224, 0.225)*10),
    ])

    train = MyDataset('dataset', 'train', train_transforms)

    train_size = int(0.7 * len(train))
    valid_size = len(train)-train_size

    train_dataset, val_dataset = random_split(train,[train_size,valid_size])

    train_loader = torch.utils.data.DataLoader(dataset=train_dataset, shuffle=True, batch_size=16, num_workers=8)
    valid_loader = torch.utils.data.DataLoader(dataset=val_dataset,shuffle=False, batch_size=8,num_workers=4)

    # step 2. load model
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model = get_model(model_name).to(device)
    torch.cuda.empty_cache()

    # step 3. prepare training tool
    criterion = nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(model.parameters(),lr=lr)
    decay_steps = (len(train_dataset)//batch_size)*nepoch
    scheduler = PolynomialLRDecay(optimizer, max_decay_steps=decay_steps, end_learning_rate=1e-6, power=0.9)
    # step 4. train model
    model = MyModelWrapper(model_name, model=model, device=device, optimizer=optimizer, criterion=criterion,scheduler=scheduler)
    model.fit(train_loader,valid_loader, nepoch=nepoch)
def run_vit_base(model_name='vit_base', lr=1e-4, batch_size=8, nepoch=100):
    download_dataset()
    # step 1. load dataset
    train_transforms = transforms.Compose([
        transforms.ToTensor(),
        transforms.RandomCrop(224),
        transforms.RandomHorizontalFlip(p=0.2),
        transforms.RandomVerticalFlip(p=0.2),
        transforms.RandomRotation(30),
        transforms.Normalize((0.485, 0.456, 0.406)*10, (0.229, 0.224, 0.225)*10),
    ])

    train = MyDataset('dataset', 'train', train_transforms)

    train_size = int(0.7 * len(train))
    valid_size = len(train)-train_size

    train_dataset, val_dataset = random_split(train,[train_size,valid_size])

    train_loader = torch.utils.data.DataLoader(dataset=train_dataset, shuffle=True, batch_size=32, num_workers=8)
    valid_loader = torch.utils.data.DataLoader(dataset=val_dataset,shuffle=False, batch_size=16,num_workers=4)

    # step 2. load model
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model = get_model(model_name).to(device)

    # step 3. prepare training tool

    criterion = nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(model.parameters(),lr=lr)
    # step 4. train model
    model = MyModelWrapper_none(model_name, model=model, device=device, optimizer=optimizer, criterion=criterion)
    # step 4. train model
    model.fit(train_loader,valid_loader, nepoch=nepoch)
def run_eff2(model_name='eff2', lr=1e-4, batch_size=32, nepoch=100): #pretrained 모델 있음

    # step 1. load dataset
    train_transforms = transforms.Compose([
        transforms.ToTensor(),
        transforms.RandomCrop(224),
        transforms.RandomHorizontalFlip(p=0.2),
        transforms.RandomVerticalFlip(p=0.2),
        transforms.RandomRotation(30),
        transforms.Normalize((0.485, 0.456, 0.406)*10, (0.229, 0.224, 0.225)*10),
    ])
    
    train = MyDataset('dataset', 'train', train_transforms)
    
    train_size = int(0.7 * len(train))
    valid_size = len(train)-train_size

    train_dataset, val_dataset = random_split(train,[train_size,valid_size])

    train_loader = torch.utils.data.DataLoader(dataset=train_dataset, shuffle=True, batch_size=32, num_workers=8)
    valid_loader = torch.utils.data.DataLoader(dataset=val_dataset,shuffle=False, batch_size=16,num_workers=4)
    
    # step 2. load model
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model = get_model(model_name).to(device)

    # step 3. prepare training tool
    criterion = nn.CrossEntropyLoss()
    optimizer = RAdam(model.parameters(), lr=lr, betas=(0.9, 0.999), weight_decay=1e-4)
    scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=10, eta_min=0)    
    model = MyModelWrapper(model_name, model=model, device=device, optimizer=optimizer, criterion=criterion, scheduler=scheduler)
    model.fit(train_loader,valid_loader, nepoch=nepoch)

def run_eff1(model_name='eff1', lr=1e-4, batch_size=32, nepoch=100): #pretrained 모델 있음

    # step 1. load dataset
    train_transforms = transforms.Compose([
        transforms.ToTensor(),
        transforms.RandomCrop(224),
        transforms.RandomHorizontalFlip(p=0.2),
        transforms.RandomVerticalFlip(p=0.2),
        transforms.RandomRotation(30),
        transforms.Normalize((0.485, 0.456, 0.406)*10, (0.229, 0.224, 0.225)*10),
    ])
    
    train = MyDataset('dataset', 'train', train_transforms)
    
    train_size = int(0.7 * len(train))
    valid_size = len(train)-train_size

    train_dataset, val_dataset = random_split(train,[train_size,valid_size])

    train_loader = torch.utils.data.DataLoader(dataset=train_dataset, shuffle=True, batch_size=32, num_workers=8)
    valid_loader = torch.utils.data.DataLoader(dataset=val_dataset,shuffle=False, batch_size=16,num_workers=4)
    
    # step 2. load model
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model = get_model(model_name).to(device)

    # step 3. prepare training tool
    criterion = nn.CrossEntropyLoss()
    scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=10, eta_min=0)    
    model = MyModelWrapper(model_name, model=model, device=device, optimizer=optimizer, criterion=criterion, scheduler=scheduler)
    model.fit(train_loader,valid_loader, nepoch=nepoch)

def run_eff0(model_name='eff0', lr=1e-4, batch_size=32, nepoch=100): #pretrained 모델 있음

    # step 1. load dataset
    train_transforms = transforms.Compose([
        transforms.ToTensor(),
        transforms.RandomCrop(224),
        transforms.RandomHorizontalFlip(p=0.2),
        transforms.RandomVerticalFlip(p=0.2),
        transforms.RandomRotation(30),
        transforms.Normalize((0.485, 0.456, 0.406)*10, (0.229, 0.224, 0.225)*10),
    ])
    
    train = MyDataset('dataset', 'train', train_transforms)
    
    train_size = int(0.7 * len(train))
    valid_size = len(train)-train_size

    train_dataset, val_dataset = random_split(train,[train_size,valid_size])

    train_loader = torch.utils.data.DataLoader(dataset=train_dataset, shuffle=True, batch_size=32, num_workers=8)
    valid_loader = torch.utils.data.DataLoader(dataset=val_dataset,shuffle=False, batch_size=16,num_workers=4)
    
    # step 2. load model
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model = get_model(model_name).to(device)

    # step 3. prepare training tool
    criterion = nn.CrossEntropyLoss()
    scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=10, eta_min=0)    
    model = MyModelWrapper(model_name, model=model, device=device, optimizer=optimizer, criterion=criterion, scheduler=scheduler)
    model.fit(train_loader,valid_loader, nepoch=nepoch)

In [41]:
class ProgressMeter(object):
    def __init__(self, num_batches, meters, prefix=""):
        self.batch_fmtstr = self._get_batch_fmtstr(num_batches)
        self.meters = meters
        self.prefix = prefix

    def display(self, batch):
        entries = [self.prefix + self.batch_fmtstr.format(batch)]
        entries += [str(meter) for meter in self.meters]
        print('\t'.join(entries))

    def _get_batch_fmtstr(self, num_batches):
        num_digits = len(str(num_batches // 1))
        fmt = '{:' + str(num_digits) + 'd}'
        return '[' + fmt + '/' + fmt.format(num_batches) + ']'


def accuracy(output, target, topk=(1,)):
    """Computes the accuracy over the k top predictions for the specified values of k"""
    with torch.no_grad():
        maxk = max(topk)
        batch_size = target.size(0)

        _, pred = output.topk(maxk, 1, True, True)
        pred = pred.t()
        correct = pred.eq(target.view(1, -1).expand_as(pred))

        res = []
        for k in topk:
            correct_k = correct[:k].reshape(-1).float().sum(0, keepdim=True)
            res.append(correct_k.mul_(100.0 / batch_size))
        return res

def get_model(model_name, n_classes=50, pretrained=True):

  if model_name=="eff7":
    model = timm.create_model("efficientnet_b7", pretrained=True, num_classes=n_classes)
  elif model_name=="eff6":
    model = timm.create_model("efficientnet_b6", pretrained=True, num_classes=n_classes)
  elif model_name=="eff5":
    model = timm.create_model("efficientnet_b5", pretrained=True, num_classes=n_classes)
  elif model_name=="eff4":
    model = timm.create_model("efficientnet_b4", pretrained=True, num_classes=n_classes)
  elif model_name=="eff3":
    model = timm.create_model("efficientnet_b3", pretrained=True, num_classes=n_classes)
  elif model_name=="eff2":
    model = timm.create_model("efficientnet_b2", pretrained=True, num_classes=n_classes)
  elif model_name=="eff1":
    model = timm.create_model("efficientnet_b1", pretrained=True, num_classes=n_classes)
  elif model_name=="eff0":
    model = timm.create_model("efficientnet_b0", pretrained=True, num_classes=n_classes)
  elif model_name=="vit_large":
    model = timm.create_model("vit_large_patch16_224", pretrained=True, num_classes=n_classes)
  elif model_name=="vit_base":
    model=timm.create_model("vit_base_patch16_224", pretrained=True,num_classes=n_classes)  
  elif model_name=="ResNet_50":
    model = timm.create_model("resnet50", pretrained=True, num_classes=n_classes)
  elif model_name=="ResNet_34":
    model = timm.create_model("resnet34", pretrained=True, num_classes=n_classes)
    
  return MnistModel_efficientb7(model=model)



def run_eff4(model_name='eff4', lr=1e-4, batch_size=32, nepoch=150): #pretrained 모델 있음
    download_dataset()
    # step 1. load dataset
    train_transforms = transforms.Compose([
        transforms.ToTensor(),
        transforms.RandomCrop(224),
        transforms.RandomHorizontalFlip(p=0.2),
        transforms.RandomVerticalFlip(p=0.2),
        transforms.RandomRotation(30),
        transforms.Normalize((0.485, 0.456, 0.406)*10, (0.229, 0.224, 0.225)*10),
    ])
    
    train = MyDataset('dataset', 'train', train_transforms)
    
    train_size = int(0.7 * len(train))
    valid_size = len(train)-train_size

    train_dataset, val_dataset = random_split(train,[train_size,valid_size])

    train_loader = torch.utils.data.DataLoader(dataset=train_dataset, shuffle=True, batch_size=32, num_workers=8)
    valid_loader = torch.utils.data.DataLoader(dataset=val_dataset,shuffle=False, batch_size=16,num_workers=4)
    
    # step 2. load model
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model = get_model(model_name).to(device)

    # step 3. prepare training tool
    criterion = nn.CrossEntropyLoss()
    optimizer = RAdam(model.parameters(), lr=lr, betas=(0.9, 0.999), weight_decay=1e-4)     
    scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=10, eta_min=0)    
    model = MyModelWrapper(model_name, model=model, device=device, optimizer=optimizer, criterion=criterion, scheduler=scheduler)
    model.fit(train_loader,valid_loader, nepoch=nepoch)

def run_eff3(model_name='eff3', lr=1e-4, batch_size=32, nepoch=100): #pretrained 모델 있음

    # step 1. load dataset
    train_transforms = transforms.Compose([
        transforms.ToTensor(),
        transforms.RandomCrop(224),
        transforms.RandomHorizontalFlip(p=0.2),
        transforms.RandomVerticalFlip(p=0.2),
        transforms.RandomRotation(30),
        transforms.Normalize((0.485, 0.456, 0.406)*10, (0.229, 0.224, 0.225)*10),
    ])
    
    train = MyDataset('dataset', 'train', train_transforms)
    
    train_size = int(0.7 * len(train))
    valid_size = len(train)-train_size

    train_dataset, val_dataset = random_split(train,[train_size,valid_size])

    train_loader = torch.utils.data.DataLoader(dataset=train_dataset, shuffle=True, batch_size=32, num_workers=8)
    valid_loader = torch.utils.data.DataLoader(dataset=val_dataset,shuffle=False, batch_size=16,num_workers=4)
    
    # step 2. load model
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model = get_model(model_name).to(device)

    # step 3. prepare training tool
    criterion = nn.CrossEntropyLoss()
    optimizer = RAdam(model.parameters(), lr=lr, betas=(0.9, 0.999), weight_decay=1e-4)    
    scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=10, eta_min=0)    
    model = MyModelWrapper(model_name, model=model, device=device, optimizer=optimizer, criterion=criterion, scheduler=scheduler)
    model.fit(train_loader,valid_loader, nepoch=nepoch)

def run_rss_50(model_name='ResNet_50', lr=1e-5, batch_size=8, nepoch=100):
    # step 1. load dataset
    train_transforms = transforms.Compose([
        transforms.ToTensor(),
        transforms.RandomCrop(224),
        transforms.RandomHorizontalFlip(p=0.2),
        transforms.RandomVerticalFlip(p=0.2),
        transforms.RandomRotation(30),
        transforms.Normalize((0.485, 0.456, 0.406)*10, (0.229, 0.224, 0.225)*10),
    ])

    train = MyDataset('dataset', 'train', train_transforms)

    train_size = int(0.7 * len(train))
    valid_size = len(train)-train_size

    train_dataset, val_dataset = random_split(train,[train_size,valid_size])

    train_loader = torch.utils.data.DataLoader(dataset=train_dataset, shuffle=True, batch_size=32, num_workers=8)
    valid_loader = torch.utils.data.DataLoader(dataset=val_dataset,shuffle=False, batch_size=16,num_workers=4)

    # step 2. load model
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model = get_model(model_name).to(device)

    # step 3. prepare training tool

    criterion = nn.CrossEntropyLoss()
    optimizer = RAdam(model.parameters(), lr=lr, betas=(0.9, 0.999), weight_decay=1e-4)    
    scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=10, eta_min=0)    
    model = MyModelWrapper(model_name, model=model, device=device, optimizer=optimizer, criterion=criterion, scheduler=scheduler)

    model.fit(train_loader,valid_loader, nepoch=nepoch)

def run_rss_34(model_name='ResNet_34', lr=1e-5, batch_size=8, nepoch=150):
    download_dataset()
    # step 1. load dataset
    train_transforms = transforms.Compose([
        transforms.ToTensor(),
        transforms.RandomCrop(224),
        transforms.RandomHorizontalFlip(p=0.2),
        transforms.RandomVerticalFlip(p=0.2),
        transforms.RandomRotation(30),
        transforms.Normalize((0.485, 0.456, 0.406)*10, (0.229, 0.224, 0.225)*10),
    ])

    train = MyDataset('dataset', 'train', train_transforms)

    train_size = int(0.7 * len(train))
    valid_size = len(train)-train_size

    train_dataset, val_dataset = random_split(train,[train_size,valid_size])

    train_loader = torch.utils.data.DataLoader(dataset=train_dataset, shuffle=True, batch_size=32, num_workers=8)
    valid_loader = torch.utils.data.DataLoader(dataset=val_dataset,shuffle=False, batch_size=16,num_workers=4)

    # step 2. load model
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model = get_model(model_name).to(device)

    # step 3. prepare training tool

    criterion = nn.CrossEntropyLoss()
    optimizer = RAdam(model.parameters(), lr=lr, betas=(0.9, 0.999), weight_decay=1e-4)   
    scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=10, eta_min=0)    
    model = MyModelWrapper(model_name, model=model, device=device, optimizer=optimizer, criterion=criterion, scheduler=scheduler)
    model.fit(train_loader,valid_loader, nepoch=nepoch)

#optimizer = torch.optim.AdamW(model.parameters(),lr=lr)
#optimizer = RAdam(model.parameters(), lr=lr, betas=(0.9, 0.999), weight_decay=1e-4)
#optimizer = torch.optim.Adam(model.parameters(),lr=lr)
run_eff4()
run_eff3()
run_eff2()
run_eff1()
run_eff0()
run_rss_50()
run_rss_34()

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 4, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


TRAIN: [0][ 1/11]	Time  0.734 ( 3.095)	Data  0.033 ( 2.356)	Loss  4.1153 ( 4.0941)	Acc@1   3.12 (  3.12)	Acc@5   9.38 (  7.81)
TRAIN: [0][ 2/11]	Time  0.677 ( 2.289)	Data  0.041 ( 1.585)	Loss  4.0152 ( 4.0678)	Acc@1   0.00 (  2.08)	Acc@5   6.25 (  7.29)
TRAIN: [0][ 3/11]	Time  0.644 ( 1.878)	Data  0.024 ( 1.195)	Loss  3.9549 ( 4.0396)	Acc@1   0.00 (  1.56)	Acc@5  15.62 (  9.38)
TRAIN: [0][ 4/11]	Time  0.643 ( 1.631)	Data  0.022 ( 0.960)	Loss  4.0059 ( 4.0328)	Acc@1   6.25 (  2.50)	Acc@5  12.50 ( 10.00)
TRAIN: [0][ 5/11]	Time  0.658 ( 1.469)	Data  0.022 ( 0.804)	Loss  3.9539 ( 4.0197)	Acc@1   3.12 (  2.60)	Acc@5  12.50 ( 10.42)
TRAIN: [0][ 6/11]	Time  0.667 ( 1.354)	Data  0.022 ( 0.692)	Loss  4.1674 ( 4.0408)	Acc@1   3.12 (  2.68)	Acc@5   3.12 (  9.38)
TRAIN: [0][ 7/11]	Time  0.662 ( 1.268)	Data  0.023 ( 0.608)	Loss  4.1510 ( 4.0546)	Acc@1   0.00 (  2.34)	Acc@5   9.38 (  9.38)
TRAIN: [0][ 8/11]	Time  0.663 ( 1.201)	Data  0.025 ( 0.544)	Loss  4.1341 ( 4.0634)	Acc@1   0.00 (  2.08)	Acc@5 

KeyboardInterrupt: ignored

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
test_transforms = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.485, 0.456, 0.406)*10, (0.229, 0.224, 0.225)*10),
])

test = MyDataset('dataset', 'test', test_transforms)
test_loader = torch.utils.data.DataLoader(dataset=test, shuffle=False, batch_size=1, num_workers=1)

In [47]:
Efficient_acc_eff1_1R = get_model('eff1')
Efficient_acc_eff1_1R.load_state_dict(torch.load('/content/drive/MyDrive/model/06-15-05-eff1.pth')['weight'])
Efficient_acc_eff1_1R.to(device)
Efficient_acc_eff1_1R.eval()

Efficient_acc_eff0_1R = get_model('eff0')
Efficient_acc_eff0_1R.load_state_dict(torch.load('/content/drive/MyDrive/model/06-30-31-eff0.pth')['weight'])
Efficient_acc_eff0_1R.to(device)
Efficient_acc_eff0_1R.eval()

Efficient_acc_eff2_1R = get_model('eff2')
Efficient_acc_eff2_1R.load_state_dict(torch.load('/content/drive/MyDrive/model/06-23-08-eff2.pth')['weight'])
Efficient_acc_eff2_1R.to(device)
Efficient_acc_eff2_1R.eval()


Efficient_acc_eff1_2R = get_model('eff1')
Efficient_acc_eff1_2R.load_state_dict(torch.load('/content/drive/MyDrive/model/07-00-22-eff1.pth')['weight'])
Efficient_acc_eff1_2R.to(device)
Efficient_acc_eff1_2R.eval()

Efficient_acc_eff0_2R = get_model('eff0')
Efficient_acc_eff0_2R.load_state_dict(torch.load('/content/drive/MyDrive/model/07-08-08-eff0 (1).pth')['weight'])
Efficient_acc_eff0_2R.to(device)
Efficient_acc_eff0_2R.eval()


Efficient_acc_eff2_2R = get_model('eff2')
Efficient_acc_eff2_2R.load_state_dict(torch.load('/content/drive/MyDrive/model/06-52-58-eff2.pth')['weight'])
Efficient_acc_eff2_2R.to(device)
Efficient_acc_eff2_2R.eval()

Efficient_acc_eff2_3R = get_model('eff2')
Efficient_acc_eff2_3R.load_state_dict(torch.load('/content/drive/MyDrive/model/07-15-18-eff2.pth')['weight'])
Efficient_acc_eff2_3R.to(device)
Efficient_acc_eff2_3R.eval()

Efficient_acc_eff1_3R = get_model('eff1')
Efficient_acc_eff1_3R.load_state_dict(torch.load('/content/drive/MyDrive/model/07-34-43-eff1.pth')['weight'])
Efficient_acc_eff1_3R.to(device)
Efficient_acc_eff1_3R.eval()

Efficient_acc_eff0_3R = get_model('eff0')
Efficient_acc_eff0_3R.load_state_dict(torch.load('/content/drive/MyDrive/model/07-42-20-eff0.pth')['weight'])
Efficient_acc_eff0_3R.to(device)
Efficient_acc_eff0_3R.eval()

Efficient_acc_eff2_4R = get_model('eff2')
Efficient_acc_eff2_4R.load_state_dict(torch.load('/content/drive/MyDrive/model/07-48-29-eff2.pth')['weight'])
Efficient_acc_eff2_4R.to(device)
Efficient_acc_eff2_4R.eval()


Efficient_acc_eff0_4R = get_model('eff0')
Efficient_acc_eff0_4R.load_state_dict(torch.load('/content/drive/MyDrive/model/08-14-56-eff0.pth')['weight'])
Efficient_acc_eff0_4R.to(device)
Efficient_acc_eff0_4R.eval()


Efficient_acc_eff4_2R = get_model('eff4')
Efficient_acc_eff4_2R.load_state_dict(torch.load('/content/drive/MyDrive/model/00-20-13-eff4.pth')['weight'])
Efficient_acc_eff4_2R.to(device)
Efficient_acc_eff4_2R.eval()


Efficient_acc_eff3_2R = get_model('eff3')
Efficient_acc_eff3_2R.load_state_dict(torch.load('/content/drive/MyDrive/model/00-36-21-eff3.pth')['weight'])
Efficient_acc_eff3_2R.to(device)
Efficient_acc_eff3_2R.eval()

ResNet_50_acc_2R = get_model('ResNet_50')
ResNet_50_acc_2R.load_state_dict(torch.load('/content/drive/MyDrive/model/00-48-17-ResNet_50.pth')['weight'])
ResNet_50_acc_2R.to(device)
ResNet_50_acc_2R.eval()


ResNet_34_acc_2R = get_model('ResNet_34')
ResNet_34_acc_2R.load_state_dict(torch.load('/content/drive/MyDrive/model/01-09-42-ResNet_34.pth')['weight'])
ResNet_34_acc_2R.to(device)
ResNet_34_acc_2R.eval()

Efficient_acc_eff4_1R = get_model('eff4')
Efficient_acc_eff4_1R.load_state_dict(torch.load('/content/drive/MyDrive/model/01-29-24-eff4.pth')['weight'])
Efficient_acc_eff4_1R.to(device)
Efficient_acc_eff4_1R.eval()


Efficient_acc_eff3_1R = get_model('eff3')
Efficient_acc_eff3_1R.load_state_dict(torch.load('/content/drive/MyDrive/model/01-48-57-eff3.pth')['weight'])
Efficient_acc_eff3_1R.to(device)
Efficient_acc_eff3_1R.eval()


ResNet_50_acc_1R = get_model('ResNet_50')
ResNet_50_acc_1R.load_state_dict(torch.load('/content/drive/MyDrive/model/01-56-48-ResNet_50.pth')['weight'])
ResNet_50_acc_1R.to(device)
ResNet_50_acc_1R.eval()

ResNet_34_acc_1R = get_model('ResNet_34')
ResNet_34_acc_1R.load_state_dict(torch.load('/content/drive/MyDrive/model/02-08-10-ResNet_34.pth')['weight'])
ResNet_34_acc_1R.to(device)
ResNet_34_acc_1R.eval()

Efficient_acc_eff4_3R = get_model('eff4')
Efficient_acc_eff4_3R.load_state_dict(torch.load('/content/drive/MyDrive/model/02-19-48-eff4.pth')['weight'])
Efficient_acc_eff4_3R.to(device)
Efficient_acc_eff4_3R.eval()


Efficient_acc_eff3_3R = get_model('eff3')
Efficient_acc_eff3_3R.load_state_dict(torch.load('/content/drive/MyDrive/model/02-28-38-eff3.pth')['weight'])
Efficient_acc_eff3_3R.to(device)
Efficient_acc_eff3_3R.eval()

ResNet_50_acc_3R = get_model('ResNet_50')
ResNet_50_acc_3R.load_state_dict(torch.load('/content/drive/MyDrive/model/02-37-47-ResNet_50.pth')['weight'])
ResNet_50_acc_3R.to(device)
ResNet_50_acc_3R.eval()


ResNet_34_acc_3R = get_model('ResNet_34')
ResNet_34_acc_3R.load_state_dict(torch.load('/content/drive/MyDrive/model/02-59-59-ResNet_34.pth')['weight'])
ResNet_34_acc_3R.to(device)
ResNet_34_acc_3R.eval()


Efficient_acc_eff4_4R = get_model('eff4')
Efficient_acc_eff4_4R.load_state_dict(torch.load('/content/drive/MyDrive/model/03-22-02-eff4.pth')['weight'])
Efficient_acc_eff4_4R.to(device)
Efficient_acc_eff4_4R.eval()


Efficient_acc_eff3_4R = get_model('eff3')
Efficient_acc_eff3_4R.load_state_dict(torch.load('/content/drive/MyDrive/model/03-44-21-eff3.pth')['weight'])
Efficient_acc_eff3_4R.to(device)
Efficient_acc_eff3_4R.eval()

ResNet_50_acc_4R = get_model('ResNet_50')
ResNet_50_acc_4R.load_state_dict(torch.load('/content/drive/MyDrive/model/04-07-31-ResNet_50.pth')['weight'])
ResNet_50_acc_4R.to(device)
ResNet_50_acc_4R.eval()


ResNet_34_acc_4R = get_model('ResNet_34')
ResNet_34_acc_4R.load_state_dict(torch.load('/content/drive/MyDrive/model/04-26-46-ResNet_34.pth')['weight'])
ResNet_34_acc_4R.to(device)
ResNet_34_acc_4R.eval()


batch_size=1

def Ensemble():
  for idx,data in enumerate(test_loader):
    images, labels = data
    images, labels = images.cuda(), labels.cuda()

    outputs = Efficient_acc_eff2_1R(images)
    outputs += Efficient_acc_eff1_1R(images)
    outputs += Efficient_acc_eff2_2R(images)
    outputs += Efficient_acc_eff1_2R(images)
    outputs += Efficient_acc_eff2_3R(images)
    outputs += Efficient_acc_eff1_3R(images)
    outputs += Efficient_acc_eff2_4R(images)

    outputs += Efficient_acc_eff4_2R(images)
    outputs += Efficient_acc_eff3_2R(images)
    outputs += ResNet_50_acc_2R(images)

    outputs += Efficient_acc_eff4_1R(images)
    outputs += Efficient_acc_eff3_1R(images)

    outputs += Efficient_acc_eff4_3R(images)
    outputs += Efficient_acc_eff3_3R(images)

    outputs += Efficient_acc_eff4_4R(images)
    outputs += Efficient_acc_eff3_4R(images)
    outputs += ResNet_50_acc_4R(images)


    _, predictions = torch.max(outputs, 1)
    predictions= torch.reshape(predictions,(-1,1))
    batch_index=batch_size*idx
    probs_array[batch_index:batch_index+batch_size]= predictions.cpu()
  return probs_array

sample_submission = pd.read_csv("https://raw.githubusercontent.com/cjfghk5697/AjouDeeplearning_Compete/main/sample_submission.csv")
probs_array = np.zeros([sample_submission.shape[0],
                              sample_submission.shape[1] -1])
probs_array=Ensemble()
sample_submission.iloc[:,1:] = probs_array
sample_submission=sample_submission.astype(int)
sample_submission.to_csv('efficientNet_Ensemble.csv',index = False) # 경로 수정 필요 


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# activation:     optimizer = torch.optim.Adam(model.parameters(),lr=lr)

# / learning scheduler: scheduler = CosineAnnealingWarmupRestarts(optimizer,
#                                          first_cycle_steps=60,
#                                         cycle_mult=1.0,
#                                          max_lr=0.005,
#                                          min_lr=0.0001,
#                                          warmup_steps=12,
#                                          gamma=1.0)
/content/drive/MyDrive/model/07-42-20-eff0.pth

Efficient_acc_eff4_2A = get_model('eff4')
Efficient_acc_eff4_2A.load_state_dict(torch.load('/content/log/best_weight/2021-09-26/12-52-55-eff4.pth')['weight'])
Efficient_acc_eff4_2A.to(device)
Efficient_acc_eff4_2A.eval()


Efficient_acc_eff3_2A = get_model('eff3')
Efficient_acc_eff3_2A.load_state_dict(torch.load('/content/log/best_weight/2021-09-26/12-59-11-eff3.pth')['weight'])
Efficient_acc_eff3_2A.to(device)
Efficient_acc_eff3_2A.eval()

Efficient_acc_eff2_2A = get_model('eff2')
Efficient_acc_eff2_2A.load_state_dict(torch.load('/content/log/best_weight/2021-09-26/12-52-55-eff4.pth')['weight'])
Efficient_acc_eff2_2A.to(device)
Efficient_acc_eff2_2A.eval()


Efficient_acc_eff1_2A = get_model('eff1')
Efficient_acc_eff1_2A.load_state_dict(torch.load('/content/log/best_weight/2021-09-26/12-59-11-eff3.pth')['weight'])
Efficient_acc_eff1_2A.to(device)
Efficient_acc_eff1_2A.eval()

Efficient_acc_eff0_2A = get_model('eff0')
Efficient_acc_eff0_2A.load_state_dict(torch.load('/content/log/best_weight/2021-09-26/12-59-11-eff3.pth')['weight'])
Efficient_acc_eff0_2A.to(device)
Efficient_acc_eff0_2A.eval()

ResNet_50_acc_2A = get_model('ResNet_50')
ResNet_50_acc_2A.load_state_dict(torch.load('/content/log/best_weight/2021-09-26/13-09-54-ResNet_50.pth')['weight'])
ResNet_50_acc_2A.to(device)
ResNet_50_acc_2A.eval()


ResNet_34_acc_2A = get_model('ResNet_34')
ResNet_34_acc_2A.load_state_dict(torch.load('/content/log/best_weight/2021-09-26/13-04-02-ResNet_34.pth')['weight'])
ResNet_34_acc_2A.to(device)
ResNet_34_acc_2A.eval()

# activation: Adam / learning       
#    decay_steps = (len(train_dataset)//batch_size)*nepoch
#    scheduler = PolynomialLRDecay(optimizer, max_decay_steps=decay_steps, end_learning_rate=1e-6, power=0.9)   



Efficient_acc_eff4_1A = get_model('eff4')
Efficient_acc_eff4_1A.load_state_dict(torch.load('/content/log/best_weight/2021-09-26/13-46-17-eff4.pth')['weight'])
Efficient_acc_eff4_1A.to(device)
Efficient_acc_eff4_1A.eval()

Efficient_acc_eff3_1A = get_model('eff3')
Efficient_acc_eff3_1A.load_state_dict(torch.load('/content/log/best_weight/2021-09-26/13-55-19-eff3.pth')['weight'])
Efficient_acc_eff3_1A.to(device)
Efficient_acc_eff3_1A.eval()

Efficient_acc_eff2_1A = get_model('eff2')
Efficient_acc_eff2_1A.load_state_dict(torch.load('/content/log/best_weight/2021-09-26/12-52-55-eff4.pth')['weight'])
Efficient_acc_eff2_1A.to(device)
Efficient_acc_eff2_1A.eval()


Efficient_acc_eff1_1A = get_model('eff1')
Efficient_acc_eff1_1A.load_state_dict(torch.load('/content/log/best_weight/2021-09-26/12-59-11-eff3.pth')['weight'])
Efficient_acc_eff1_1A.to(device)
Efficient_acc_eff1_1A.eval()

Efficient_acc_eff0_1A = get_model('eff0')
Efficient_acc_eff0_1A.load_state_dict(torch.load('/content/log/best_weight/2021-09-26/12-59-11-eff3.pth')['weight'])
Efficient_acc_eff0_1A.to(device)
Efficient_acc_eff0_1A.eval()


Vit_base_acc_1A=get_model('vit_base')
Vit_base_acc_1A.load_state_dict(torch.load('/content/log/best_weight/2021-09-26/19-19-06-vit_base.pth')['weight'])
Vit_base_acc_1A.to(device)
Vit_base_acc_1A.eval()

ResNet_50_acc_1A = get_model('ResNet_50')
ResNet_50_acc_1A.load_state_dict(torch.load('/content/log/best_weight/2021-09-26/13-58-57-ResNet_50.pth')['weight'])
ResNet_50_acc_1A.to(device)
ResNet_50_acc_1A.eval()

ResNet_34_acc_1A = get_model('ResNet_34')
ResNet_34_acc_1A.load_state_dict(torch.load('/content/log/best_weight/2021-09-26/14-09-17-ResNet_34.pth')['weight'])
ResNet_34_acc_1A.to(device)
ResNet_34_acc_1A.eval()


# Optimizer: Adam learning scheduler:  scheduler =  scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=10, eta_min=0)

Efficient_acc_eff4_3A = get_model('eff4')
Efficient_acc_eff4_3A.load_state_dict(torch.load('/content/log/best_weight/2021-09-26/14-21-05-eff4.pth')['weight'])
Efficient_acc_eff4_3A.to(device)
Efficient_acc_eff4_3A.eval()

Efficient_acc_eff3_3A = get_model('eff3')
Efficient_acc_eff3_3A.load_state_dict(torch.load('/content/log/best_weight/2021-09-26/14-30-48-eff3.pth')['weight'])
Efficient_acc_eff3_3A.to(device)
Efficient_acc_eff3_3A.eval()

Efficient_acc_eff2_3A = get_model('eff2')
Efficient_acc_eff2_3A.load_state_dict(torch.load('/content/log/best_weight/2021-09-26/12-52-55-eff4.pth')['weight'])
Efficient_acc_eff2_3A.to(device)
Efficient_acc_eff2_3A.eval()


Efficient_acc_eff1_3A = get_model('eff1')
Efficient_acc_eff1_3A.load_state_dict(torch.load('/content/log/best_weight/2021-09-26/12-59-11-eff3.pth')['weight'])
Efficient_acc_eff1_3A.to(device)
Efficient_acc_eff1_3A.eval()

Efficient_acc_eff0_3A = get_model('eff0')
Efficient_acc_eff0_3A.load_state_dict(torch.load('/content/log/best_weight/2021-09-26/12-59-11-eff3.pth')['weight'])
Efficient_acc_eff0_3A.to(device)
Efficient_acc_eff0_3A.eval()

Vit_base_acc_3A=get_model('vit_base')
Vit_base_acc_3A.load_state_dict(torch.load('/content/log/best_weight/2021-09-26/20-14-38-vit_base.pth')['weight'])
Vit_base_acc_3A.to(device)
Vit_base_acc_3A.eval()

ResNet_50_acc_3A = get_model('ResNet_50')
ResNet_50_acc_3A.load_state_dict(torch.load('/content/log/best_weight/2021-09-26/14-34-36-ResNet_50.pth')['weight'])
ResNet_50_acc_3A.to(device)
ResNet_50_acc_3A.eval()

ResNet_34_acc_3A = get_model('ResNet_34')
ResNet_34_acc_3A.load_state_dict(torch.load('/content/log/best_weight/2021-09-26/14-44-32-ResNet_34.pth')['weight'])
ResNet_34_acc_3A.to(device)
ResNet_34_acc_3A.eval()

In [ ]:
# optimizer = RAdam(model.parameters(), lr=lr, betas=(0.9, 0.999), weight_decay=1e-4)
# learning scheduler: scheduler = CosineAnnealingWarmupRestarts(optimizer,
#                                          first_cycle_steps=60,
#                                         cycle_mult=1.0,
#                                          max_lr=0.005,
#                                          min_lr=0.0001,
#                                          warmup_steps=12,
#                                          gamma=1.0)
Efficient_acc_eff4_2R = get_model('eff4')
Efficient_acc_eff4_2R.load_state_dict(torch.load('/content/log/best_weight/2021-09-26/14-54-33-eff4.pth')['weight'])
Efficient_acc_eff4_2R.to(device)
Efficient_acc_eff4_2R.eval()

Efficient_acc_eff3_2R = get_model('eff3')
Efficient_acc_eff3_2R.load_state_dict(torch.load('/content/log/best_weight/2021-09-26/14-58-23-eff3.pth')['weight'])
Efficient_acc_eff3_2R.to(device)
Efficient_acc_eff3_2R.eval()

Efficient_acc_eff2_2R = get_model('eff2')
Efficient_acc_eff2_2R.load_state_dict(torch.load('/content/log/best_weight/2021-09-26/12-52-55-eff4.pth')['weight'])
Efficient_acc_eff2_2R.to(device)
Efficient_acc_eff2_2R.eval()


Efficient_acc_eff1_2R = get_model('eff1')
Efficient_acc_eff1_2R.load_state_dict(torch.load('/content/log/best_weight/2021-09-26/12-59-11-eff3.pth')['weight'])
Efficient_acc_eff1_2R.to(device)
Efficient_acc_eff1_2R.eval()

Efficient_acc_eff0_2R = get_model('eff0')
Efficient_acc_eff0_2R.load_state_dict(torch.load('/content/log/best_weight/2021-09-26/12-59-11-eff3.pth')['weight'])
Efficient_acc_eff0_2R.to(device)
Efficient_acc_eff0_2R.eval()


ResNet_50_acc_2R = get_model('ResNet_50')
ResNet_50_acc_2R.load_state_dict(torch.load('/content/log/best_weight/2021-09-26/15-00-58-ResNet_50.pth')['weight'])
ResNet_50_acc_2R.to(device)
ResNet_50_acc_2R.eval()

ResNet_34_acc_2R = get_model('ResNet_34')
ResNet_34_acc_2R.load_state_dict(torch.load('/content/log/best_weight/2021-09-26/15-08-28-ResNet_34.pth')['weight'])
ResNet_34_acc_2R.to(device)
ResNet_34_acc_2R.eval()

# optimizer = RAdam(model.parameters(), lr=lr, betas=(0.9, 0.999), weight_decay=1e-4)
#    decay_steps = (len(train_dataset)//batch_size)*nepoch
#    scheduler = PolynomialLRDecay(optimizer, max_decay_steps=decay_steps, end_learning_rate=1e-6, power=0.9)   

Efficient_acc_eff4_1R = get_model('eff4')
Efficient_acc_eff4_1R.load_state_dict(torch.load('/content/log/best_weight/2021-09-26/15-15-04-eff4.pth')['weight'])
Efficient_acc_eff4_1R.to(device)
Efficient_acc_eff4_1R.eval()

Efficient_acc_eff3_1R = get_model('eff3')
Efficient_acc_eff3_1R.load_state_dict(torch.load('/content/log/best_weight/2021-09-26/15-30-07-eff3.pth')['weight'])
Efficient_acc_eff3_1R.to(device)
Efficient_acc_eff3_1R.eval()

Efficient_acc_eff2_1R = get_model('eff2')
Efficient_acc_eff2_1R.load_state_dict(torch.load('/content/log/best_weight/2021-09-26/12-52-55-eff4.pth')['weight'])
Efficient_acc_eff2_1R.to(device)
Efficient_acc_eff2_1R.eval()


Efficient_acc_eff1_1R = get_model('eff1')
Efficient_acc_eff1_1R.load_state_dict(torch.load('/content/log/best_weight/2021-09-26/12-59-11-eff3.pth')['weight'])
Efficient_acc_eff1_1R.to(device)
Efficient_acc_eff1_1R.eval()

Efficient_acc_eff0_1R = get_model('eff0')
Efficient_acc_eff0_1R.load_state_dict(torch.load('/content/log/best_weight/2021-09-26/12-59-11-eff3.pth')['weight'])
Efficient_acc_eff0_1R.to(device)
Efficient_acc_eff0_1R.eval()


Vit_base_acc_1R=get_model('vit_base')
Vit_base_acc_1R.load_state_dict(torch.load('/content/log/best_weight/2021-09-26/19-29-02-vit_base.pth')['weight'])
Vit_base_acc_1R.to(device)
Vit_base_acc_1R.eval()

ResNet_50_acc_1R = get_model('ResNet_50')
ResNet_50_acc_1R.load_state_dict(torch.load('/content/log/best_weight/2021-09-26/15-39-47-ResNet_50.pth')['weight'])
ResNet_50_acc_1R.to(device)
ResNet_50_acc_1R.eval()

ResNet_34_acc_1R = get_model('ResNet_34')
ResNet_34_acc_1R.load_state_dict(torch.load('/content/log/best_weight/2021-09-26/15-59-19-ResNet_34.pth')['weight'])
ResNet_34_acc_1R.to(device)
ResNet_34_acc_1R.eval()

# optimizer = RAdam(model.parameters(), lr=lr, betas=(0.9, 0.999), weight_decay=1e-4)
  
Efficient_acc_eff4_3R = get_model('eff4')
Efficient_acc_eff4_3R.load_state_dict(torch.load('/content/log/best_weight/2021-09-26/16-17-34-eff4.pth')['weight'])
Efficient_acc_eff4_3R.to(device)
Efficient_acc_eff4_3R.eval()

Efficient_acc_eff3_3R = get_model('eff3')
Efficient_acc_eff3_3R.load_state_dict(torch.load('/content/log/best_weight/2021-09-26/16-33-59-eff3.pth')['weight'])
Efficient_acc_eff3_3R.to(device)
Efficient_acc_eff3_3R.eval()

Efficient_acc_eff2_3R = get_model('eff2')
Efficient_acc_eff2_3R.load_state_dict(torch.load('/content/log/best_weight/2021-09-26/12-52-55-eff4.pth')['weight'])
Efficient_acc_eff2_3R.to(device)
Efficient_acc_eff2_3R.eval()


Efficient_acc_eff1_3R = get_model('eff1')
Efficient_acc_eff1_3R.load_state_dict(torch.load('/content/log/best_weight/2021-09-26/12-59-11-eff3.pth')['weight'])
Efficient_acc_eff1_3R.to(device)
Efficient_acc_eff1_3R.eval()

Efficient_acc_eff0_3R = get_model('eff0')
Efficient_acc_eff0_3R.load_state_dict(torch.load('/content/log/best_weight/2021-09-26/12-59-11-eff3.pth')['weight'])
Efficient_acc_eff0_3R.to(device)
Efficient_acc_eff0_3R.eval()

Vit_base_acc_3R=get_model('vit_base')
Vit_base_acc_3R.load_state_dict(torch.load('/content/log/best_weight/2021-09-26/20-10-19-vit_base.pth')['weight'])
Vit_base_acc_3R.to(device)
Vit_base_acc_3R.eval()

ResNet_50_acc_3R = get_model('ResNet_50')
ResNet_50_acc_3R.load_state_dict(torch.load('/content/log/best_weight/2021-09-26/16-43-55-ResNet_50.pth')['weight'])
ResNet_50_acc_3R.to(device)
ResNet_50_acc_3R.eval()

ResNet_34_acc_3R = get_model('ResNet_34')
ResNet_34_acc_3R.load_state_dict(torch.load('/content/log/best_weight/2021-09-26/17-02-47-ResNet_34.pth')['weight'])
ResNet_34_acc_3R.to(device)
ResNet_34_acc_3R.eval()

In [ ]:
# activation:AdamW     optimizer = torch.optim.AdamW(model.parameters(),lr=lr)
# learning scheduler: scheduler = CosineAnnealingWarmupRestarts(optimizer,
#                                          first_cycle_steps=60,
#                                         cycle_mult=1.0,
#                                          max_lr=0.005,
#                                          min_lr=0.0001,
#                                          warmup_steps=12,
#                                          gamma=1.0)

Efficient_acc_eff4_2W = get_model('eff4')
Efficient_acc_eff4_2W.load_state_dict(torch.load('/content/log/best_weight/2021-09-26/17-26-55-eff4.pth')['weight'])
Efficient_acc_eff4_2W.to(device)
Efficient_acc_eff4_2W.eval()

Efficient_acc_eff3_2W = get_model('eff3')
Efficient_acc_eff3_2W.load_state_dict(torch.load('/content/log/best_weight/2021-09-26/17-30-02-eff3.pth')['weight'])
Efficient_acc_eff3_2W.to(device)
Efficient_acc_eff3_2W.eval()

Efficient_acc_eff2_2W = get_model('eff2')
Efficient_acc_eff2_2W.load_state_dict(torch.load('/content/log/best_weight/2021-09-26/12-52-55-eff4.pth')['weight'])
Efficient_acc_eff2_2W.to(device)
Efficient_acc_eff2_2W.eval()


Efficient_acc_eff1_2W = get_model('eff1')
Efficient_acc_eff1_2W.load_state_dict(torch.load('/content/log/best_weight/2021-09-26/12-59-11-eff3.pth')['weight'])
Efficient_acc_eff1_2W.to(device)
Efficient_acc_eff1_2W.eval()

Efficient_acc_eff0_2W = get_model('eff0')
Efficient_acc_eff0_2W.load_state_dict(torch.load('/content/log/best_weight/2021-09-26/12-59-11-eff3.pth')['weight'])
Efficient_acc_eff0_2W.to(device)
Efficient_acc_eff0_2W.eval()


ResNet_50_acc_2W = get_model('ResNet_50')
ResNet_50_acc_2W.load_state_dict(torch.load('/content/log/best_weight/2021-09-26/17-35-06-ResNet_50.pth')['weight'])
ResNet_50_acc_2W.to(device)
ResNet_50_acc_2W.eval()

ResNet_34_acc_2W = get_model('ResNet_34')
ResNet_34_acc_2W.load_state_dict(torch.load('/content/log/best_weight/2021-09-26/17-44-26-ResNet_34.pth')['weight'])
ResNet_34_acc_2W.to(device)
ResNet_34_acc_2W.eval()

# activation: AdamW / learning scheduler
#    decay_steps = (len(train_dataset)//batch_size)*nepoch
#    scheduler = PolynomialLRDecay(optimizer, max_decay_steps=decay_steps, end_learning_rate=1e-6, power=0.9)   

Efficient_acc_eff4_1W = get_model('eff4')
Efficient_acc_eff4_1W.load_state_dict(torch.load('/content/log/best_weight/2021-09-26/17-50-30-eff4.pth')['weight'])
Efficient_acc_eff4_1W.to(device)
Efficient_acc_eff4_1W.eval()

Efficient_acc_eff3_1W = get_model('eff3')
Efficient_acc_eff3_1W.load_state_dict(torch.load('/content/log/best_weight/2021-09-26/18-01-10-eff3.pth')['weight'])
Efficient_acc_eff3_1W.to(device)
Efficient_acc_eff3_1W.eval()

Efficient_acc_eff2_1W = get_model('eff2')
Efficient_acc_eff2_1W.load_state_dict(torch.load('/content/log/best_weight/2021-09-26/12-52-55-eff4.pth')['weight'])
Efficient_acc_eff2_1W.to(device)
Efficient_acc_eff2_1W.eval()


Efficient_acc_eff1_1W = get_model('eff1')
Efficient_acc_eff1_1W.load_state_dict(torch.load('/content/log/best_weight/2021-09-26/12-59-11-eff3.pth')['weight'])
Efficient_acc_eff1_1W.to(device)
Efficient_acc_eff1_1W.eval()

Efficient_acc_eff0_1W = get_model('eff0')
Efficient_acc_eff0_1W.load_state_dict(torch.load('/content/log/best_weight/2021-09-26/12-59-11-eff3.pth')['weight'])
Efficient_acc_eff0_1W.to(device)
Efficient_acc_eff0_1W.eval()

Vit_base_acc_1W=get_model('vit_base')
Vit_base_acc_1W.load_state_dict(torch.load('/content/log/best_weight/2021-09-26/19-33-48-vit_base.pth')['weight'])
Vit_base_acc_1W.to(device)
Vit_base_acc_1W.eval()

ResNet_50_acc_1W = get_model('ResNet_50')
ResNet_50_acc_1W.load_state_dict(torch.load('/content/log/best_weight/2021-09-26/18-04-45-ResNet_50.pth')['weight'])
ResNet_50_acc_1W.to(device)
ResNet_50_acc_1W.eval()

ResNet_34_acc_1W = get_model('ResNet_34')
ResNet_34_acc_1W.load_state_dict(torch.load('/content/log/best_weight/2021-09-26/18-29-04-ResNet_34.pth')['weight'])
ResNet_34_acc_1W.to(device)
ResNet_34_acc_1W.eval()

# Optimizer: AdamW learning scheduler:  scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=10, eta_min=0)

Efficient_acc_eff4_3W = get_model('eff4')
Efficient_acc_eff4_3W.load_state_dict(torch.load('/content/log/best_weight/2021-09-26/18-29-51-eff4.pth')['weight'])
Efficient_acc_eff4_3W.to(device)
Efficient_acc_eff4_3W.eval()

Efficient_acc_eff3_3W = get_model('eff3')
Efficient_acc_eff3_3W.load_state_dict(torch.load('/content/log/best_weight/2021-09-26/18-38-14-eff3.pth')['weight'])
Efficient_acc_eff3_3W.to(device)
Efficient_acc_eff3_3W.eval()

Efficient_acc_eff2_3W = get_model('eff2')
Efficient_acc_eff2_3W.load_state_dict(torch.load('/content/log/best_weight/2021-09-26/12-52-55-eff4.pth')['weight'])
Efficient_acc_eff2_3W.to(device)
Efficient_acc_eff2_3W.eval()

Efficient_acc_eff1_3W = get_model('eff1')
Efficient_acc_eff1_3W.load_state_dict(torch.load('/content/log/best_weight/2021-09-26/12-59-11-eff3.pth')['weight'])
Efficient_acc_eff1_3W.to(device)
Efficient_acc_eff1_3W.eval()

Efficient_acc_eff0_3W = get_model('eff0')
Efficient_acc_eff0_3W.load_state_dict(torch.load('/content/log/best_weight/2021-09-26/12-59-11-eff3.pth')['weight'])
Efficient_acc_eff0_3W.to(device)
Efficient_acc_eff0_3W.eval()

Vit_base_acc_3W=get_model('vit_base')
Vit_base_acc_3W.load_state_dict(torch.load('/content/log/best_weight/2021-09-26/20-03-39-vit_base.pth')['weight'])
Vit_base_acc_3W.to(device)
Vit_base_acc_3W.eval()

ResNet_50_acc_3W = get_model('ResNet_50')
ResNet_50_acc_3W.load_state_dict(torch.load('/content/log/best_weight/2021-09-26/18-42-40-ResNet_50.pth')['weight'])
ResNet_50_acc_3W.to(device)
ResNet_50_acc_3W.eval()

ResNet_34_acc_3W = get_model('ResNet_34')
ResNet_34_acc_3W.load_state_dict(torch.load('/content/log/best_weight/2021-09-26/18-53-12-ResNet_34.pth')['weight'])
ResNet_34_acc_3W.to(device)
ResNet_34_acc_3W.eval()


In [ ]:
batch_size=1

def Ensemble():
  for idx,data in enumerate(test_loader):
    images, labels = data
    images, labels = images.cuda(), labels.cuda()
    

    outputs = Efficient_acc_eff4_1A(images)
    outputs += Efficient_acc_eff3_1A(images)
    outputs += Efficient_acc_eff2_1A(images)
    outputs += Efficient_acc_eff1_1A(images)
    outputs += Efficient_acc_eff0_1A(images)
    outputs += ResNet_50_acc_1A(images)
    outputs += ResNet_34_acc_1A(images)

    outputs += Efficient_acc_eff4_2A(images)
    outputs += Efficient_acc_eff3_2A(images)
    outputs += Efficient_acc_eff2_2A(images)
    outputs += Efficient_acc_eff1_2A(images)
    outputs += Efficient_acc_eff0_2A(images)
    outputs += ResNet_50_acc_2A(images)
    outputs += ResNet_34_acc_2A(images)

    outputs += Efficient_acc_eff4_3A(images)
    outputs += Efficient_acc_eff3_3A(images)
    outputs += Efficient_acc_eff2_3A(images)
    outputs += Efficient_acc_eff1_3A(images)
    outputs += Efficient_acc_eff0_3A(images)
    outputs += ResNet_50_acc_3A(images)
    outputs += ResNet_34_acc_3A(images)

    outputs += Efficient_acc_eff4_1R(images)
    outputs += Efficient_acc_eff3_1R(images)
    outputs += Efficient_acc_eff2_1R(images)
    outputs += Efficient_acc_eff1_1R(images)
    outputs += Efficient_acc_eff0_1R(images)
    outputs += ResNet_50_acc_1R(images)
    outputs += ResNet_34_acc_1R(images)

    outputs += Efficient_acc_eff4_2R(images)
    outputs += Efficient_acc_eff3_2R(images)
    outputs += Efficient_acc_eff2_2R(images)
    outputs += Efficient_acc_eff1_2R(images)
    outputs += Efficient_acc_eff0_2R(images)
    outputs += ResNet_50_acc_2R(images)
    outputs += ResNet_34_acc_2R(images)

    outputs += Efficient_acc_eff4_3R(images)
    outputs += Efficient_acc_eff3_3R(images)
    outputs += Efficient_acc_eff2_3R(images)
    outputs += Efficient_acc_eff1_3R(images)
    outputs += Efficient_acc_eff0_3R(images)
    outputs += ResNet_50_acc_3R(images)
    outputs += ResNet_34_acc_3R(images)

    outputs += Efficient_acc_eff4_1W(images)
    outputs += Efficient_acc_eff3_1W(images)
    outputs += Efficient_acc_eff2_1W(images)
    outputs += Efficient_acc_eff1_1W(images)
    outputs += Efficient_acc_eff0_1W(images)
    outputs += ResNet_50_acc_1W(images)
    outputs += ResNet_34_acc_1W(images)

    outputs += Efficient_acc_eff4_2W(images)
    outputs += Efficient_acc_eff3_2W(images)
    outputs += Efficient_acc_eff2_2W(images)
    outputs += Efficient_acc_eff1_2W(images)
    outputs += Efficient_acc_eff0_2W(images)
    outputs += ResNet_50_acc_2W(images)
    outputs += ResNet_34_acc_2W(images)

    outputs += Efficient_acc_eff4_3W(images)
    outputs += Efficient_acc_eff3_3W(images)
    outputs += Efficient_acc_eff2_3W(images)
    outputs += Efficient_acc_eff1_3W(images)
    outputs += Efficient_acc_eff0_3W(images)
    outputs += ResNet_50_acc_3W(images)
    outputs += ResNet_34_acc_3W(images)

    _, predictions = torch.max(outputs, 1)
    predictions= torch.reshape(predictions,(-1,1))
    batch_index=batch_size*idx
    probs_array[batch_index:batch_index+batch_size]= predictions.cpu()
  return probs_array

sample_submission = pd.read_csv("https://raw.githubusercontent.com/cjfghk5697/AjouDeeplearning_Compete/main/sample_submission.csv")
probs_array = np.zeros([sample_submission.shape[0],
                              sample_submission.shape[1] -1])
probs_array=Ensemble()
sample_submission.iloc[:,1:] = probs_array
sample_submission=sample_submission.astype(int)
sample_submission.to_csv('efficientNet_Ensemble.csv',index = False) # 경로 수정 필요 